# Run epigenomic models

```{note}
Please only use the following versions:
`python`: 3.8.16
`pacmap`: 0.7.0
`lightgbm`: 3.3.5
`scikit-learn`: 1.2.2
```

## Where the data at

In [4]:
import pandas as pd

mount = '/mnt/e/'

input_path = mount + 'MethylScore_v2/Intermediate_Files/'
output_path = mount + 'MethylScore_v2/Processed_Data/'

nanopore_path = mount + 'nanopore_processed/pacmap/'

sample_name = 'uf_hembank_1852'

## Load data

In [5]:
df_nanopore = pd.read_pickle(
    nanopore_path + sample_name + '_pacmap_bvalues.pkl').sort_index()

print(
f' Nanopore dataset (df_nanopore) contains {df_nanopore.shape[1]} \
columns (5mC nucleotides/probes) and {df_nanopore.shape[0]} rows (samples).')

df_discovery = pd.read_pickle(
    input_path+'3308samples_333059cpgs_withbatchcorrection_bvalues.pkl').sort_index().iloc[:,1:][df_nanopore.columns]

print(
f' Discovery dataset (df_discovery) contains {df_discovery.shape[1]} \
columns (5mC nucleotides/probes) and {df_discovery.shape[0]-1} rows (samples).')


 Nanopore dataset (df_nanopore) contains 331886 columns (5mC nucleotides/probes) and 1 rows (samples).
 Discovery dataset (df_discovery) contains 331886 columns (5mC nucleotides/probes) and 3307 rows (samples).


## Reduce dimensionality with PaCMAP

### Set hyperparameters, fit, and save

In [6]:
import pacmap
print('PaCMAP version:', pacmap.__version__)

components_list = [2, 5]
for components in components_list:
    
    reducer = pacmap.PaCMAP(n_components=components, 
                            n_neighbors=10, 
                            MN_ratio=0.4, 
                            FP_ratio=16.0, 
                            lr=0.1, 
                            num_iters=5000,
                            random_state=42,
                            save_tree=True)

    # Project the high dimensional dataset into a low-dimensional embedding
    embedding_training = reducer.fit_transform(df_discovery.to_numpy(dtype='float16'))

    # Save reducer
    pacmap.save(reducer, nanopore_path + f'{sample_name}_pacmap_{components}d_model_al_atlas')

    # Create column names
    cols = ['PaCMAP '+ str(i+1) + f' of {components}' for i in range(components)]

    # Turn embedding into dataframe
    df_embedding = pd.DataFrame(embedding_training, columns=cols, index=df_discovery.index).to_pickle(
        nanopore_path+f'{sample_name}_pacmap_{components}d_embedding.pkl')


PaCMAP version: 0.7.0


/home/marchi/projects/AL_atlas_notebooks/.venv_py38/lib/python3.8/site-packages/pacmap/pacmap.py:819: UserWarning: Warning: random state is set to 42
  warnings.warn(f'Warning: random state is set to {_RANDOM_STATE}')


The PaCMAP instance is successfully saved at /mnt/e/nanopore_processed/pacmap/uf_hembank_1852_pacmap_2d_model_al_atlas.pkl, and the Annoy Index is saved at /mnt/e/nanopore_processed/pacmap/uf_hembank_1852_pacmap_2d_model_al_atlas.ann.
To load the instance again, please do `pacmap.load(/mnt/e/nanopore_processed/pacmap/uf_hembank_1852_pacmap_2d_model_al_atlas)`.


/home/marchi/projects/AL_atlas_notebooks/.venv_py38/lib/python3.8/site-packages/pacmap/pacmap.py:819: UserWarning: Warning: random state is set to 42
  warnings.warn(f'Warning: random state is set to {_RANDOM_STATE}')


The PaCMAP instance is successfully saved at /mnt/e/nanopore_processed/pacmap/uf_hembank_1852_pacmap_5d_model_al_atlas.pkl, and the Annoy Index is saved at /mnt/e/nanopore_processed/pacmap/uf_hembank_1852_pacmap_5d_model_al_atlas.ann.
To load the instance again, please do `pacmap.load(/mnt/e/nanopore_processed/pacmap/uf_hembank_1852_pacmap_5d_model_al_atlas)`.


### Apply models to new data

In [7]:

def apply_pacmap_model_to_new_data(df, components):

    # Load reducer
    reducer = pacmap.load(nanopore_path + f'{sample_name}_pacmap_{components}d_model_al_atlas')

    # Project the high dimensional dataset into existing embedding space and return the embedding.
    embedding = reducer.transform(df.to_numpy(dtype='float16'))

    # Create column names
    cols = ['PaCMAP '+ str(i+1) + f' of {components}' for i in range(components)]

    # Turn embedding into dataframe
    df_embedding = pd.DataFrame(embedding, columns=cols, index=df.index)

    return df_embedding

# Apply pacmap model to discovery dataset
train_2d = pd.read_pickle(nanopore_path+sample_name+'_pacmap_2d_embedding.pkl')
train_5d = pd.read_pickle(nanopore_path+sample_name+'_pacmap_5d_embedding.pkl')

# # Apply pacmap model to validation dataset
test_2d = apply_pacmap_model_to_new_data(df_nanopore, components=2)
test_5d = apply_pacmap_model_to_new_data(df_nanopore, components=5)

### Merge results with clinical data

In [8]:
# Join 2d and 5d
train = train_2d.join(train_5d)
test = test_2d.join(test_5d)

# Concatenate train and test
train_test = pd.concat([train, test])

# Read clinical data
clinical_data = pd.read_excel(input_path+'clinical_data.xlsx', index_col=0)

# Join train_test with clinical data
df = train_test.join(clinical_data)

train_test_5d = pd.concat([train_5d, test_5d])
train_test_2d = pd.concat([train_2d, test_2d])

## Preprocess data for classifiers

### Select samples Px

In [9]:

# Drop the samples with missing labels for the selected column
df_px = df[~df['Vital Status'].isna()]

df_px2 = df_px.copy()

# # Exclude the `Blood Derived Normal`and `Bone Marrow Normal` from `Sample Type`
# df_px2 = df_px[~df_px['Sample Type'].isin([
#                                             'Relapse', 'Recurrent Blood Derived Cancer - Bone Marrow',
#                                             'Recurrent Blood Derived Cancer - Peripheral Blood',
#                                             'Blood Derived Normal', 'Bone Marrow Normal'
# ])]

# print the number of samples dropped and the amount remaining
print(df.shape[0]-df_px2.shape[0], 'samples removed.'\
, df_px2.shape[0], 'samples remaining.')

1465 samples removed. 1844 samples remaining.


### Select samples Dx

In [10]:
# drop the samples with missing labels for the ELN AML 2022 Diagnosis
df_dx = df[~df['WHO 2022 Diagnosis'].isna()]

# exclude the classes with fewer than 10 samples
df_dx2 = df_dx[~df_dx['WHO 2022 Diagnosis'].isin([
                                       'MPAL with t(v;11q23.3)/KMT2A-r',
                                       'B-ALL with hypodiploidy',
                                       'AML with t(16;21); FUS::ERG',
                                       'AML with t(9;22); BCR::ABL1'
                                       ])]

# print the number of samples dropped and the amount remaining
print(df.shape[0]-df_dx2.shape[0], 'samples removed.'\
, df_dx2.shape[0], 'samples remaining.')

864 samples removed. 2445 samples remaining.


### Define X and y

In [11]:
def custom_train_test_split(df, feature_columns, target_column, split_column):

    X = df[feature_columns].to_numpy(dtype='float16')
    y = df[target_column].to_numpy()

    train_mask = df[split_column] == 'Train Sample'
    test_mask = df[split_column] == 'Test Sample'

    X_train, X_test = X[train_mask], X[test_mask]
    y_train, y_test = y[train_mask], y[test_mask]

    print('X_train shape:', X_train.shape, 'X_test shape:', X_test.shape)

    return X_train, X_test, y_train, y_test

# Execution
X_train_dx, X_test_dx, y_train_dx, y_test_dx = custom_train_test_split(df_dx2, test_5d.columns,'WHO 2022 Diagnosis', 'Train-Test')
X_train_px, X_test_px, y_train_px, y_test_px = custom_train_test_split(df_px2, test_5d.columns,'Vital Status', 'Train-Test')


X_train shape: (2445, 5) X_test shape: (0, 5)
X_train shape: (1844, 5) X_test shape: (0, 5)


## Benchmark classifiers

In [12]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
import joblib
import sys
sys.path.append('../')

def benchmark_classifier(X_train, y_train):

    param_grid = {
    # 'num_leaves': [ 5, 6, 7, 8, 9, 10], # number of leaves in full tree
    # 'max_depth': [3,4,5,6,7,8,9,10],  # maximum depth of a tree
    # 'learning_rate': [0.01],  # learning rate
    # 'n_estimators': [50, 100, 500],  # number of trees (or rounds)
    'reg_alpha': [1],  # L1 regularization
    'reg_lambda': [1],  # L2 regularization
    'class_weight': ['balanced'],  # weights associated with classes
    }

    # Initialize the LGBM Classifier with regularization
    lgbm = LGBMClassifier(random_state=42, n_jobs=-1) 
                          
    # Perform grid search with stratified cross-validation
    grid_search = GridSearchCV(lgbm, param_grid, cv=10, n_jobs=-1,
                               scoring='roc_auc_ovr_weighted')

    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)
    print(f"Best parameters: {grid_search.best_params_}")

    # Get the best model
    clf = grid_search.best_estimator_

    return clf

# Benchmark, train
lgbm_dx_model = benchmark_classifier(X_train_dx, y_train_dx)
lgbm_px_model = benchmark_classifier(X_train_px, y_train_px)

Best parameters: {'class_weight': 'balanced', 'reg_alpha': 1, 'reg_lambda': 1}
Best parameters: {'class_weight': 'balanced', 'reg_alpha': 1, 'reg_lambda': 1}


## Apply supervised models

In [13]:
def save_predictions(df, classifier, model_name):

    # ignore sklearn warnings
    import warnings
    warnings.filterwarnings('ignore')

    # Select necessary columns
    df_features = df.copy()

    # Predict using the selected columns
    predictions = classifier.predict(df_features)

    # Predict probabilities using the selected columns
    probabilities = classifier.predict_proba(df_features)

    # Convert predictions to a Series with the same index as df_features
    predictions_series = pd.Series(predictions, index=df_features.index, name=model_name)

    # Convert probabilities to a DataFrame with the same index as df_features and the same columns as the classes
    probabilities_df = pd.DataFrame(probabilities, index=df_features.index, columns=classifier.classes_).round(3)

    # Add " - predict_proba" to the column names
    probabilities_df.columns ='P(' + probabilities_df.columns + ')'

    # Transform classes of the predictions into integers based on unique values in the classes
    probabilities_df[model_name + '_int'] = predictions_series.map({c: i for i, c in enumerate(classifier.classes_)})

    # Join predictions with the original DataFrame (already indexed)
    df_joined = predictions_series.to_frame().join(probabilities_df)

    return df_joined

# Execution
df_pred_px = save_predictions(df=train_test_5d, classifier=lgbm_px_model, model_name='AML Epigenomic Risk')
df_pred_dx = save_predictions(df=train_test_5d, classifier=lgbm_dx_model, model_name='AL Epigenomic Phenotype')

# Map the classes to more desirable labels (low and high risk)
df_pred_px['AML Epigenomic Risk'] = df_pred_px['AML Epigenomic Risk'].map({'Alive': 'Low', 'Dead': 'High'})
df_pred_px = df_pred_px.rename(columns={'P(Alive)': 'AML Epigenomic Risk P(Low Risk)', 'P(Dead)': 'AML Epigenomic Risk P(High Risk)'})

# Join predictions with clinical data
df_combined = train_test_2d.join(train_test_5d).join(df_pred_px).join(df_pred_dx)


df_combined.iloc[-1:,:][['AML Epigenomic Risk', 'AML Epigenomic Risk P(High Risk)', 'AL Epigenomic Phenotype', f'P({df_combined.iloc[-1:,:]["AL Epigenomic Phenotype"].item()})']]

,AML Epigenomic Risk,AML Epigenomic Risk P(High Risk),AL Epigenomic Phenotype,P(AML with NUP98-fusion)
uf_hembank_1852,High,0.786,AML with NUP98-fusion,0.83


In [96]:
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, FactorRange, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

# Prepare the data
data = df_pred_dx.iloc[-1, 1:-1]

# Sort columns by values
data = data.sort_values().iloc[-5:]

source = ColumnDataSource(data=dict(diagnoses=data.index.tolist(),
                                    probabilities=data.values.tolist()))

# Set the output to notebook
output_notebook()

# Create the figure
p = figure(y_range=FactorRange(*data.index.tolist()), height=200, 
           title="Predicted Probabilities of WHO 2022 Diagnosis",)

# Add a horizontal bar chart
p.hbar(y='diagnoses', right='probabilities', source=source, 
       fill_color=factor_cmap('diagnoses', palette="Viridis256",
                              factors=data.index.tolist()))

# Set the axis labels
p.xaxis.axis_label = "Predicted Probability"
p.yaxis.axis_label = "WHO 2022 Diagnosis"

# Add hover tool
hover = HoverTool()
hover.tooltips = [("Diagnosis", "@diagnoses"), ("Probability", "@probabilities{0.000}")]
p.add_tools(hover)

# hide logo
p.toolbar.logo = None

# # Set other options
p.x_range.start = 0
p.x_range.end = 1
p.y_range.range_padding = 0.1
p.yaxis.major_label_orientation = 0  # Adjusted to make labels horizontal
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

# Display the plot
show(p)

Loading BokehJS ...

## EWASCox-Lasso

In [82]:
import math
import sys
sys.path.append('../')
from source.cox_lasso import *

raw_coefs = pd.read_csv(output_path + 'multivariate_cox_lasso/ewas_cog_os_raw_coefs_newrisk.csv', index_col=0)

mean_coefs = set_cutoff(coefs=raw_coefs,threshold=0.99)

df_validation = df_nanopore[mean_coefs.index]

df_validation_transformed = df_validation.replace(1, 0.999).replace(0, 0.001)

def beta2m(val):
    '''Transfrom beta-values into m-values'''
    return math.log2(val/(1-val))

x_test_m = df_validation_transformed.apply(np.vectorize(beta2m))

def standardize_data(df, reference_df):
    """Standardize data using mean and standard deviation of reference dataset"""

    # Keep only columns that are in both datasets
    reference_df = reference_df.loc[:, df.columns]

    # Standardize data
    df_z = (df - reference_df.mean()) / reference_df.std()

    return df_z

# Read top CpGs selected from previous code file (univariate cox-ph EWAS)
ewas_top_cpgs = pd.read_csv(output_path+'ewas_dmr/ewas_top_cpgs_os.csv', index_col=0)

# Standardize data
x_test_m_z = standardize_data(df= x_test_m, reference_df= ewas_top_cpgs)

score_name = 'EWASCox_OS_48CpGs'

df_test, threshold = generate_coxph_score(coef_mean=mean_coefs,
                                        x=x_test_m_z,
                                        df=df_validation_transformed,
                                        score_name=score_name,
                                        train_test=0.4934,
                                        rpart_outcome='os.time')

df_validation_transformed[['EWASCox_OS_48CpGs','EWASCox_OS_48CpGs Categorical']]

Continuous score cut at the value of 0.4934


name,EWASCox_OS_48CpGs,EWASCox_OS_48CpGs Categorical
uf_hembank_1852,-0.722272,Low


## Save results

In [83]:
try:
    df_nanopore = df_combined.join(df_validation_transformed[['EWASCox_OS_48CpGs','EWASCox_OS_48CpGs Categorical']])
except:
    df_nanopore = df_combined.copy()

# Merge with clinical data
df_nanopore = df_nanopore.join(clinical_data)

# Update the 'Train-Test' column of the last row
df_nanopore.iat[-1, df_nanopore.columns.get_loc('Train-Test')] = 'Long-read ONTseq'

# Update the 'Clinical Trial' column of the last row
df_nanopore.iat[-1, df_nanopore.columns.get_loc('Clinical Trial')] = 'UF HemBank'

# Update the 'Patient_ID' column of the last row
df_nanopore.iat[-1, df_nanopore.columns.get_loc('Patient_ID')] = sample_name

# Save
df_nanopore.to_pickle(output_path + sample_name + '_processed.pkl')

# print save message
print(f'Processed data for {sample_name} saved as {output_path + sample_name + "_processed.pkl"}')

Processed data for uf_hembank_1852 saved as /mnt/e/MethylScore_v2/Processed_Data/uf_hembank_1852_processed.pkl


## Watermark

In [ ]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [ ]:
# watermark with all libraries used in this notebook
%watermark -v -p numpy,pandas,pacmap,sklearn,lightgbm -a Francisco_Marchi@Lamba_Lab_UF -d -m

Author: Francisco_Marchi@Lamba_Lab_UF

Python implementation: CPython
Python version       : 3.8.18
IPython version      : 8.12.2

numpy   : 1.24.3
pandas  : 2.0.2
pacmap  : 0.7.0
sklearn : 1.2.2
lightgbm: 3.3.5

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.133.1-microsoft-standard-WSL2
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit



```{note}
Updating to python 3.10 caused substantial delays in lightgbm, so please use the following versions:
`python`: 3.8.16
`pacmap`: 0.7.0
`lightgbm`: 3.3.5
`scikit-learn`: 1.2.2
```